In [55]:
import pandas as pd
import random
import os
import glob


In [56]:

#################################################
# PART 1: Find and load all CSV files
#################################################
print("STEP 1: Finding and loading CSV files")
csv_files = glob.glob('*.csv')
print(f"Found {len(csv_files)} CSV files: {csv_files}")

if not csv_files:
    print("No CSV files found in the current directory.")
    exit()

csv_dataframes = {}
for filename in csv_files:
    try:
        csv_dataframes[filename] = pd.read_csv(filename)
        print(f"Loaded {filename}")
    except Exception as e:
        print(f"Error loading {filename}: {e}")



STEP 1: Finding and loading CSV files
Found 8 CSV files: ['use_summary.csv', 'module_scraped_with_content.csv', 'sbert_detailed.csv', 'bloom_detailed.csv', 'sbert_summary.csv', 'course_scraped_with_lo.csv', 'use_detailed.csv', 'bloom_summary.csv']
Loaded use_summary.csv
Loaded module_scraped_with_content.csv
Loaded sbert_detailed.csv
Loaded bloom_detailed.csv
Loaded sbert_summary.csv
Loaded course_scraped_with_lo.csv
Loaded use_detailed.csv
Loaded bloom_summary.csv


In [57]:

#################################################
# PART 2: Extract and randomly select URLs from specified CSV
#################################################
print("\nSTEP 2: Extracting URLs from course_scraped_with_lo.csv")
# Define the specific source file
source_file = "course_scraped_with_lo.csv"

# Check if specified source file exists in the loaded dataframes
if source_file not in csv_dataframes:
    print(f"Error: {source_file} not found in directory.")
    exit()

# Check if 'url' column exists in the source file
if 'url' not in csv_dataframes[source_file].columns:
    print(f"Error: No 'url' column found in {source_file}.")
    exit()

# Extract URLs from the specified source
source_urls = csv_dataframes[source_file]['url'].tolist()
# Remove duplicates from the URL list
source_urls = list(dict.fromkeys(source_urls))
print(f"Found {len(source_urls)} unique URLs in {source_file}")

if not source_urls:
    print("No URLs found in the source CSV file.")
    exit()

# Select random URLs
print("\nSTEP 3: Randomly selecting URLs")
num_urls_to_select = min(10, len(source_urls))  # Select urls
selected_urls = random.sample(source_urls, num_urls_to_select)
print(f"Randomly selected {num_urls_to_select} URLs from {source_file}:")
for i, url in enumerate(selected_urls, 1):
    print(f"{i}. {url}")


STEP 2: Extracting URLs from course_scraped_with_lo.csv
Found 1993 unique URLs in course_scraped_with_lo.csv

STEP 3: Randomly selecting URLs
Randomly selected 10 URLs from course_scraped_with_lo.csv:
1. https://www.coursera.org/learn/uva-darden-agile-analytics
2. https://www.coursera.org/learn/craft-of-plot
3. https://www.coursera.org/learn/dsp1
4. https://www.coursera.org/learn/investment-banking-mergers-acquisitions-ipos
5. https://www.coursera.org/learn/google-docs
6. https://www.coursera.org/learn/healthcare-data-literacy
7. https://www.coursera.org/learn/video-game-story
8. https://www.coursera.org/learn/user-interface-in-game-design
9. https://www.coursera.org/learn/java-object-oriented-programming
10. https://www.coursera.org/learn/web-design-strategy


In [58]:

#################################################
# PART 3: Search for matches in all CSV files
#################################################
print("\nSTEP 4: Searching for matches in all CSVs")
# Dictionary to store results by filename
results_by_file = {}
match_found = False

# Track processed rows to avoid duplicates
processed_rows = {}  # Dictionary to track processed rows by URL and file

for url in selected_urls:
    print(f"\nSearching for URL: {url}")
    url_found = False

    for filename, df in csv_dataframes.items():
        if 'url' in df.columns:
            # Perform the search
            matches = df[df['url'] == url]

            if not matches.empty:
                url_found = True
                match_found = True

                # Initialize tracking for this file if needed
                if filename not in processed_rows:
                    processed_rows[filename] = set()

                # Initialize this file's results list if needed
                if filename not in results_by_file:
                    results_by_file[filename] = []

                # Filter and process unique matches
                unique_matches = 0
                for idx, row in matches.iterrows():
                    # Create a hashable representation of the row for duplicate checking
                    row_hash = hash(tuple(row.items()))

                    # Only process if we haven't seen this exact row before
                    if row_hash not in processed_rows[filename]:
                        processed_rows[filename].add(row_hash)
                        unique_matches += 1
                        results_by_file[filename].append(row.to_dict())


    if not url_found:
        print(f"  Warning: URL {url} was not found in any CSV")


STEP 4: Searching for matches in all CSVs

Searching for URL: https://www.coursera.org/learn/uva-darden-agile-analytics

Searching for URL: https://www.coursera.org/learn/craft-of-plot

Searching for URL: https://www.coursera.org/learn/dsp1

Searching for URL: https://www.coursera.org/learn/investment-banking-mergers-acquisitions-ipos

Searching for URL: https://www.coursera.org/learn/google-docs

Searching for URL: https://www.coursera.org/learn/healthcare-data-literacy

Searching for URL: https://www.coursera.org/learn/video-game-story

Searching for URL: https://www.coursera.org/learn/user-interface-in-game-design

Searching for URL: https://www.coursera.org/learn/java-object-oriented-programming

Searching for URL: https://www.coursera.org/learn/web-design-strategy


In [59]:

#################################################
# PART 4: Create Excel output with sheets in specific order
#################################################
print("\nSTEP 5: Creating Excel output with sheets in specified order")
if match_found:
    output_filename = "url_selection_results.xlsx"

    # Define the desired sheet order
    sheet_order = [
        "Summary",  # Summary sheet is always first
        "course_scraped_with_lo",
        "module_scraped_with_content",
        "sbert_summary",
        "sbert_detailed",
        "use_summary",
        "use_detailed",
        "bloom_summary",
        "bloom_detailed"
    ]

    # Create a new Excel writer object
    with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
        # Create the summary sheet first
        summary_df = pd.DataFrame({'Selected URLs': selected_urls})
        summary_df.to_excel(writer, sheet_name='Summary', index=False)
        print(f"Created sheet 'Summary' with {len(selected_urls)} selected URLs")

        # Create sheets in the specified order
        for sheet_name in sheet_order[1:]:  # Skip 'Summary' as we already created it
            # Find the corresponding CSV file
            matching_files = [f for f in results_by_file.keys() if sheet_name in f]

            if matching_files:
                filename = matching_files[0]  # Use the first matching file
                matches = results_by_file[filename]

                if matches:  # Only create sheets if there are matches
                    # Convert list of dictionaries to DataFrame
                    result_df = pd.DataFrame(matches)

                    # Double-check for any remaining duplicates and remove them
                    result_df = result_df.drop_duplicates()

                    # Write DataFrame to Excel sheet
                    sheet_name_excel = sheet_name[:31]  # Excel sheet names max 31 chars
                    result_df.to_excel(writer, sheet_name=sheet_name_excel, index=False)
                    print(f"Created sheet '{sheet_name_excel}' with {len(result_df)} unique matches")

        # Add any remaining files that weren't in the specified order
        remaining_files = [f for f in results_by_file.keys() if not any(sheet in f for sheet in sheet_order[1:])]
        for filename in remaining_files:
            matches = results_by_file[filename]

            if matches:  # Only create sheets if there are matches
                # Convert list of dictionaries to DataFrame
                result_df = pd.DataFrame(matches)

                # Double-check for any remaining duplicates and remove them
                result_df = result_df.drop_duplicates()

                # Write DataFrame to Excel sheet
                sheet_name = os.path.splitext(filename)[0][:31]  # Excel sheet names max 31 chars
                result_df.to_excel(writer, sheet_name=sheet_name, index=False)
                print(f"Created sheet '{sheet_name}' with {len(result_df)} unique matches")

    print(f"\nResults saved to {output_filename}")
else:
    print("No matching results found for any of the selected URLs.")


STEP 5: Creating Excel output with sheets in specified order
Created sheet 'Summary' with 10 selected URLs
Created sheet 'course_scraped_with_lo' with 11 unique matches
Created sheet 'module_scraped_with_content' with 45 unique matches
Created sheet 'sbert_summary' with 11 unique matches
Created sheet 'sbert_detailed' with 102 unique matches
Created sheet 'use_summary' with 11 unique matches
Created sheet 'use_detailed' with 102 unique matches
Created sheet 'bloom_summary' with 10 unique matches
Created sheet 'bloom_detailed' with 102 unique matches

Results saved to url_selection_results.xlsx
